Importujemy najpierw potrzebne pakiety

In [80]:
import numpy as np
import scipy.io.wavfile
import scipy.signal

def wczytywanie_piosenki(nazwa_piosenki = "song.txt"):

    """
    wczytuje zawartosc pliku song.txt do macierzy numpy-owej
    
    arg:
        str: nazwa_piosenki - nazwa pliku tekstowego wraz z rozszerzeniem (zakladam, tak jak w opisie 
                              projektu, ze bedziemy korzystac z domyslnej wartosci, tj. "song.txt", wiec
                              nie sprawdzam poprawnosci typu tego argumentu; problemem z brakiem tego pliku
                              etc. zajmie sie juz python, wyrzucajac komunikat, wiec tez pomijam)
    
    wyjscie:
        numpy.ndarray (str: U2): song - macierz wymiaru 1 na k (ilosc wierszy w pliku song.txt), zawierajaca
                                        kolejne wiersze z wejsciowego pliku zapisane jako stringi
                                    
        
    """
    
    import numpy as np

    song = np.genfromtxt(nazwa_piosenki,dtype='str')
    
    return song

a = wczytywanie_piosenki()

In [146]:
def wczytywanie_sciezek(lista_trackow):
    """
    
    wczytuje na podstawie pliku "song.txt" (lub innego, ktory definiował lista_tackow) zawartosc plikow 
    "trackXY.txt" do jednej macierzy numpy-owej 
    
    arg:
        numpy.ndarray (str: U2): lista_trackow - macierz wymiaru 1 na k, zawierajaca zapisane jako napisy,
                                                 numery kolejnych sciezek do odtworzenia
    
    wyjscie:
        tuple: numpy.ndarray (str: U2): piosenka - macierz, zawierajaca jako wiersze kolejne cwiercnuty utworu,
                                            w kazdym wierszu informacja czy dany sample ma zagrac czy nie,
                                            zapisana jako napis (albo numer sampla albo --)
               bool: dobre_trackusie - zmienna kontrolujaca poprawnosc wczytywanych trackow (metrum i ilosc
                                       sampli)
                                            
    """
    
    import numpy as np
    
    # zapisujemy unikalne numery trackow, ktore maja byc wczytane
    jakie_sciezki = np.unique(lista_trackow)

    # slownik, ktorego kolejne elementy to unikalne sciezki (key) wraz z ich zawartoscia (value)
    sciezki = {}
    
    # zmienna do sprawdzenia czy zachowane jest odpowiednie metrum (liczba wierszy podzielna przez 4) w kazdej
    # ze sciezek, jesli nie, to zwracana macierz bedzie zerowa
    czy_4_na_4 = True
    
    # zmienna do sprawdzenia czy w kazdym tracku uzyto tyle samo sampli, jesli nie, zwracana macierz bedzie
    # zerowa
    czy_tyle_samo_sampli = True
    
    # lista zawierajaca ilosc uzytych sampli w kolejnych trackach
    ile_sampli = []
    
    # cale to definiowanie slownika i szukanie unikalnych trackow po to by raz wczytac zawartosc uzywanych
    # trackow a potem sie do niej odwolywac, a nie wczytywac za kazdym razem od nowa zawartosc trackow
    
    for ktora_sciezka in jakie_sciezki: # w iteratorze bierzemy napisy "01" "02" itd. stringi!!!
        
        # tworzymy napis z nazwa pliku tracka, np. "track01.txt"
        plik = ''.join(['track',ktora_sciezka,'.txt'])
        # wczytujemy zawartosc danego tracka do odpowiednio nazwanego elementu w slowniku sciezki
        sciezki[ktora_sciezka] = np.genfromtxt(plik, dtype='str')
        # dodajemy ilosc kanalow danego tracka do listy
        ile_sampli.append(sciezki[ktora_sciezka].shape[1])
        # sprawdzamy podzielnosc wierdzy danego tracka przez 4, jesli nie zachodzi, drukujemy odpowiedni
        # komunikat i zmieniamy wartosc zmiennej logicznej czy_4_na_4 na False
        if(sciezki[ktora_sciezka].shape[0] % 4 != 0):
            print("Plik: ", plik, " nie ma podzielnej przez 4 liczby wierszy")
            czy_4_na_4 = False

    
    # sprawdzamy czy ilosc uzytych sampli jest taka sama w kazdym tracku
    if(len(set(ile_sampli))!=1): # set-em wyciagamy unikalne wartosci i sprawdzamy czy jest tylko jedna taka
        print("Rozna liczba uzytych sampli w trackach")
        czy_tyle_samo_sampli = False
    
    # jedna zmienna odpowiadajaca za poprawnosc trackow
    dobre_trackusie = czy_4_na_4 and czy_tyle_samo_sampli
    
    # jesli metrum i ilosc sampli sie zgadza zapisujemy zawartosc sciezek w piosence do numpy.ndarraya
    if(dobre_trackusie):
 
            # do zapisu poslugujemy sie lista i funkcja append na liscie, gdyz jest to bardziej efektywne
            # niz append dla numpy.ndarraya, gdy nie znamy jego ostatecznych rozmiarow
            piosenka = []
            
             # tym razem iterujemy po kolejnych trackach z lista_trackow, nie po unikalnnych nazwach trackow
            for ktora_sciezka in lista_trackow:
                piosenka.append(sciezki[ktora_sciezka]) #dodajemy odpowiednia wartosc ze slownika sciezki

            #tworzymy z listy obiekt typu numpy.ndarray
            piosenka = np.array(piosenka)
            # obiekt ten zawiera kilka podmacierzy, wiec tworzymy z niego 1 macierz
            piosenka = np.vstack(piosenka)
    # gdy nie ma odpowiedniego metrum lub ilosci sampli zwracamy zerowa macierz      
    else: 
        piosenka = np.array([0])

    return piosenka, dobre_trackusie 

pios, k = wczytywanie_sciezek(a)

In [ ]:
def wczytywanie_ustawien(plik_konfiguracyjny = "defs.txt"):
    """
    asserty 
    na razie zakladam, ze wszystkie parametry beda liczbami
    
    wczytywanie pliku z ustawieniami (pliku defs.txt)
    
    arg:
        nazwa pliku konfiguracyjnego, typ: string
        
    wyjscie:
        wartosci parametrow zapisane w slowniku, typ: dict
    
    """
    import re
    # wczytuje zawartowsc pliku (bez pierwszej i ostatniej linijki, jeden wiersz wyjsciowej macierzy, zawiera
    # nazwe parametru i jego wartosc, jako oddzielne elementy, zapisane jako stringi)
    ustawienia = np.genfromtxt(plik_konfiguracyjny, dtype = str,skip_header=1,skip_footer=1, delimiter=":")
    # tworze slownik, ktory bedzie przechowywal wartosci
    parametry = {}
    # tworze slownik
    # pozbywam się "" z key i przerabiam value na inty
    if len(ustawienia) == 2:
        parametry[re.sub('"','',ustawienia[0])] = int(ustawienia[1])
    else:
        for l in ustawienia:
        
            parametry[re.sub('"','',l[0])] = int(l[1])
    
    # zwracam gotowy slownik
    return parametry
b=wczytywanie_ustawien()

In [116]:


# NOWE PRZERÓBKA
def tworzenie_piosenki(macierz_piosenki, czy_pelna = True, bpmy = 120):
    """
    asserty 
    glowna funkcja
    """
    
    if(czy_pelna == False):
        print("Nie utworzono piosenki")
        return None
    else:
    
    
        t_cwiercnuty = 60 / bpmy
        n = macierz_piosenki.shape[0]
        kanaly = macierz_piosenki.shape[1]

        import numpy as np
        import scipy.io.wavfile
        import scipy.signal

        # utwor
        T = np.linspace(0, n*t_cwiercnuty, 44100*n*t_cwiercnuty)

        calosc = np.zeros((1,))
        for wiersz in range(0, 10): # ZMIEN TU NA N ZAMIAST 10

            sample = []
            dlugosci = []

            for i in range(0, kanaly):

                if macierz_piosenki[wiersz,i] == '--':
                    sample.append(np.zeros((1,1)))    
                else:
                    plik = ''.join(['sample', macierz_piosenki[wiersz,i],'.wav']) 
                    frekw, t = scipy.io.wavfile.read(plik)
                    sample.append(t)
                    sample[i] = np.mean(sample[i], axis=1)/32767 # robi mono
                    sample[i] = np.int16(sample[i]/max(np.abs(sample[i]))*32767)
                    dlugosci.append(sample[i].shape[0])

            maksik = max(dlugosci)
            pusty = np.int16(np.zeros((len(sample), maksik)))
            for k in range(0, len(sample)):
                pusty[k][0:sample[k].shape[0]] = sample[k]

            cwiercnuta = sum(pusty)
            if (wiersz*t_cwiercnuty*44100 + maksik) > len(T):
                T[wiersz*t_cwiercnuty*44100:(wiersz*t_cwiercnuty*44100 + maksik)]= cwiercnuta[0:len(T[wiersz*t_cwiercnuty*44100:(wiersz*t_cwiercnuty*44100 + maksik)])]
            else:
                T[wiersz*t_cwiercnuty*44100:(wiersz*t_cwiercnuty*44100 + maksik)]= cwiercnuta





            #print(len(T[wiersz*t_cwiercnuty*44100:(wiersz*t_cwiercnuty*44100 + maksik)]))
            #print(len(cwiercnuta))
            #print(type(T[wiersz*t_cwiercnuty*44100:(wiersz*t_cwiercnuty*44100 + maksik)]))

        T= np.array(T, dtype=np.int16)
        return T

pios, k = wczytywanie_sciezek(a)
wierszyk = tworzenie_piosenki(pios, k, b['bpm'])

# jak w to wcisnac bmpki?
wierszyk

/home/rexamine/anaconda/lib/python3.4/site-packages/scipy/io/wavfile.py:172: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


array([  54, -124,  203, ...,   45,   45,   46], dtype=int16)

In [27]:
wierszyk = np.array(wierszyk/2,dtype=np.int16)

In [2]:
import numpy as np
import scipy.io.wavfile
import scipy.signal

# dac ifa ze jak wierszyk = none to nic nie robić

scipy.io.wavfile.write('test1.wav', 
                           44100,
                       wierszyk
                        )


In [3]:
!!aplay /home/rexamine/Desktop/test1.wav

["Playing WAVE '/home/rexamine/Desktop/test1.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Mono"]

In [4]:
frekw, t = scipy.io.wavfile.read("sample01.wav")

/home/rexamine/anaconda/lib/python3.4/site-packages/scipy/io/wavfile.py:172: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)
